In [2]:
import os
from dotenv import load_dotenv

In [5]:
raw = os.getenv("GROQ_API_KEY")

In [7]:
from src.config_loader import load_config

In [14]:
config = load_config()

In [15]:
config.get("providers", {}).get("embedding")

'google'